In [1]:
import pandas as pd
import os, sys
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    TextTransform
)


C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
computer_name1 = 'gilbe'
computer_name2 = 'Gilberto-BE'

ROOT_PATH = f'c:/Users/{computer_name1}/Documents/TokyoData'

In [3]:
stock_list = pd.read_csv(f'{ROOT_PATH}/stock_list.csv').drop('Close', axis=1)
print(stock_list.info())
print()
display(stock_list.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  IssuedShares            4121 non-null   float64
 13  MarketCapitalization    4121 non-null   float64
 14  Universe0               4417 non-null   

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,7.263200e+04,3.216145e+09,False


In [4]:



'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
# train_df['Date'] = pd.to_datetime(train_df['Date']) 
# train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
# train_options['Date'] = pd.to_datetime(train_options['Date'])
# train_options.set_index('Date', inplace=True)

train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
# train_financials['Date'] = pd.to_datetime(train_financials['Date'])
# train_financials.set_index('Date', inplace=True)

train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


display(train_df.head())
display(train_df.tail())
display(train_df.info())
print('.' * 20)
print()



print('OPTIONS')
display(train_options.head())
display(train_options.info())
print('.' * 50)
print()



print('FINANCIALS')
display(train_financials.head())

FIN_COLS_CONT = [
        'NetSales', 'EquityToAssetRatio', 'TotalAssets', 'Profit', 
        'OperatingProfit', 'EarningsPerShare', 'Equity', 
        'BookValuePerShare', 'ResultDividendPerShare1stQuarter', 
        'ResultDividendPerShare2ndQuarter', 'ResultDividendPerShare3rdQuarter',
        'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual'
        ]
display(train_financials[FIN_COLS_CONT].head())
display(train_financials.info())
print('.' * 50)
print()



print('TRADES')
display(train_trades.head())
display(train_trades.info())
print('.' * 50)
print()


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098
2332530,20211203_9997,2021-12-03,9997,690.0,711.0,686.0,696.0,381100,1.0,NaN,False,0.018414


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


None

....................

OPTIONS


...

..................................................

TRADES


,Date,StartDate,EndDate,Section,TotalSales,TotalPurchases,TotalTotal,TotalBalance,ProprietarySales,ProprietaryPurchases,...,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
0,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 1712 non-null   object 
 1   StartDate                            765 non-null    object 
 2   EndDate                              765 non-null    object 
 3   Section                              765 non-null    object 
 4   TotalSales                           765 non-null    float64
 5   TotalPurchases                       765 non-null    float64
 6   TotalTotal                           765 non-null    float64
 7   TotalBalance                         765 non-null    float64
 8   ProprietarySales                     765 non-null    float64
 9   ProprietaryPurchases                 765 non-null    float64
 10  ProprietaryTotal                     765 non-null    float64
 11  ProprietaryBalance            

None

..................................................



# Merging data

### Ideas
Train some stocks from some industries

In [5]:
stock_list[['SecuritiesCode', 'MarketCapitalization']]

,SecuritiesCode,MarketCapitalization
0,1301,3.365911e+10
1,1305,7.621831e+12
2,1306,1.641739e+13
3,1308,7.671945e+12
4,1309,3.216145e+09
...,...,...
4412,9994,2.654474e+10
4413,9995,1.083499e+10
4414,9996,1.361913e+10
4415,9997,6.894633e+10


In [6]:
display(train_financials[train_financials['SecuritiesCode'] == 1301].head())
train_financials[train_financials['SecuritiesCode'] == 1301].shape

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
2781,2.016113e+13,20170210_1301,2017-02-10,1301.0,2017-02-10,11:30:00,1.486694e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,199.94,NaN,False,True,False,False,False,10928283,425455,10503022
3940,2.017022e+13,20170217_1301,2017-02-17,1301.0,2017-02-17,15:00:00,1.487311e+09,ForecastRevision,2017-03-31,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6892,2.017033e+13,20170511_1301,2017-05-11,1301.0,2017-05-11,11:30:00,1.494470e+09,FYFinancialStatements_Consolidated_JP,2017-03-31,FY,...,257.07,NaN,False,True,False,False,False,10928283,425510,10502960
11274,2.017062e+13,20170804_1301,2017-08-04,1301.0,2017-08-04,13:00:00,1.501819e+09,1QFinancialStatements_Consolidated_JP,2017-06-30,1Q,...,257.07,NaN,False,False,False,False,False,10928283,425510,10502773
15877,2.017091e+13,20171106_1301,2017-11-06,1301.0,2017-11-06,11:30:00,1.509935e+09,2QFinancialStatements_Consolidated_JP,2017-09-30,2Q,...,257.07,NaN,False,False,False,False,False,10928283,422090,10503343


(23, 45)

In [7]:
merged_data = stock_list.merge(train_df, on='SecuritiesCode')

merged_data = merged_data.merge(train_financials, on=['SecuritiesCode', 'Date'], how='left')
display(merged_data.head())
display(merged_data.info())

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335601 entries, 0 to 2335600
Data columns (total 69 columns):
 #   Column                                                                        Dtype  
---  ------                                                                        -----  
 0   SecuritiesCode                                                                int64  
 1   EffectiveDate                                                                 int64  
 2   Name                                                                          object 
 3   Section/Products                                                              object 
 4   NewMarketSegment                                                              object 
 5   33SectorCode                                                                  object 
 6   33SectorName                                                                  object 
 7   17SectorCode                                                   

None

In [8]:
print(merged_data[merged_data['SecuritiesCode'] == 1301].shape)
display(merged_data[merged_data['SecuritiesCode'] == 1301])
print(merged_data[merged_data['SecuritiesCode'] == 1301]['NumberOfTreasuryStockAtTheEndOfFiscalYear'].unique()) 

(1204, 69)


,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1200,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1201,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[nan '425455' '425510' '422090' '411773' '144712' '143521' '109062'
 '109112' '109196' '109203' '187735' '184241' '170699' '170774' '170681'
 '170467']


In [9]:
print(merged_data[merged_data['SecuritiesCode'] == 1301].columns) 

Index(['SecuritiesCode', 'EffectiveDate', 'Name', 'Section/Products',
       'NewMarketSegment', '33SectorCode', '33SectorName', '17SectorCode',
       '17SectorName', 'NewIndexSeriesSizeCode', 'NewIndexSeriesSize',
       'TradeDate', 'IssuedShares', 'MarketCapitalization', 'Universe0',
       'RowId', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Target',
       'DisclosureNumber', 'DateCode', 'DisclosedDate', 'DisclosedTime',
       'DisclosedUnixTime', 'TypeOfDocument', 'CurrentPeriodEndDate',
       'TypeOfCurrentPeriod', 'CurrentFiscalYearStartDate',
       'CurrentFiscalYearEndDate', 'NetSales', 'OperatingProfit',
       'OrdinaryProfit', 'Profit', 'EarningsPerShare', 'TotalAssets', 'Equity',
       'EquityToAssetRatio', 'BookValuePerShare',
       'ResultDividendPerShare1stQuarter', 'ResultDividendPerShare2ndQuarter',
       'ResultDividendPerShare3rdQuarter',
       'ResultDividendPerShareFiscalYearEnd', 'R

In [10]:
merged_data[merged_data['SecuritiesCode'] == 1301].shape

(1204, 69)

In [11]:
stock_list['Section/Products'].unique()

array(['First Section (Domestic)', 'ETFs/ ETNs',
       'JASDAQ(Standard / Domestic)', 'JASDAQ(Growth/Domestic)',
       'Mothers (Domestic)', 'Second Section(Domestic)',
       'First Section (Foreign)',
       'REIT, Venture Funds, Country Funds and Infrastructure Funds',
       'JASDAQ(Standard / Foreign)', 'Mothers (Foreign)',
       'Second Section(Foreign)'], dtype=object)

In [12]:
stock_list['NewMarketSegment'].unique()

array(['Prime Market', nan, 'Standard Market', 'Growth Market',
       'Prime Market (Foreign Stock)', 'Standard Market (Foreign Stock)',
       'Growth Market (Foreign Stock)'], dtype=object)

In [13]:
stock_list['IssuedShares'].unique()

array([1.09282830e+07, 3.63463562e+09, 7.91771845e+09, ...,
       9.15264000e+06, 9.72444720e+07, 3.42469620e+07])

In [14]:
print(stock_list.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  IssuedShares            4121 non-null   float64
 13  MarketCapitalization    4121 non-null   float64
 14  Universe0               4417 non-null   

In [15]:
print(stock_list.shape)

(4417, 15)


# Text transform data on whole dataset
* Make merges necessary 
* Do text transform on whole merged data

In [16]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
print(stock_list['NewMarketSegment'])
print()
token_list = list(stock_list['NewMarketSegment'])

print('token_list[:20]:')
print(token_list[:10])
txt_trans = TextTransform(token_list)
print(txt_trans.transform()[:20])



0          Prime Market
1                   NaN
2                   NaN
3                   NaN
4                   NaN
             ...       
4412    Standard Market
4413       Prime Market
4414    Standard Market
4415       Prime Market
4416                NaN
Name: NewMarketSegment, Length: 4417, dtype: object

token_list[:20]:
['Prime Market', nan, nan, nan, nan, nan, nan, nan, nan, nan]
[2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2]


In [17]:
print(train_df.head(2))
print(train_df.tail(2))

print(train_df.info())
print('.' * 20)
print()

g_by_tgt = train_df.sort_values(['Date', 'Target']).groupby(['Date', 'SecuritiesCode'])

           RowId        Date  SecuritiesCode    Open    High     Low   Close  \
0  20170104_1301  2017-01-04            1301  2734.0  2755.0  2730.0  2742.0   
1  20170104_1332  2017-01-04            1332   568.0   576.0   563.0   571.0   

    Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag    Target  
0    31400               1.0               NaN            False  0.000730  
1  2798500               1.0               NaN            False  0.012324  
                 RowId        Date  SecuritiesCode    Open    High     Low  \
2332529  20211203_9994  2021-12-03            9994  2388.0  2396.0  2380.0   
2332530  20211203_9997  2021-12-03            9997   690.0   711.0   686.0   

          Close  Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  \
2332529  2389.0    6500               1.0               NaN            False   
2332530   696.0  381100               1.0               NaN            False   

           Target  
2332529  0.009098  
2332530  0.0184

In [18]:
g_by_tgt.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
1733,20170104_9474,2017-01-04,9474,2149.0,2156.0,2072.0,2119.0,325800,1.0,NaN,False,-0.069874
1856,20170104_9983,2017-01-04,9983,42480.0,43330.0,42450.0,43290.0,648100,1.0,NaN,False,-0.067387
1136,20170104_6875,2017-01-04,6875,2587.0,2611.0,2562.0,2591.0,192200,1.0,NaN,False,-0.062112
1852,20170104_9974,2017-01-04,9974,4515.0,4610.0,4500.0,4600.0,26400,1.0,NaN,False,-0.059276
1078,20170104_6707,2017-01-04,6707,519.0,543.0,517.0,539.0,2203000,1.0,NaN,False,-0.057514
...,...,...,...,...,...,...,...,...,...,...,...,...
2330836,20211203_3031,2021-12-03,3031,1191.0,1193.0,1155.0,1155.0,1651500,1.0,NaN,False,0.119069
2331997,20211203_7809,2021-12-03,7809,6060.0,6430.0,5830.0,6410.0,46300,1.0,NaN,False,0.136667
2331721,20211203_6779,2021-12-03,6779,1558.0,1582.0,1489.0,1546.0,519200,1.0,NaN,False,0.159624
2330587,20211203_1873,2021-12-03,1873,342.0,352.0,340.0,352.0,84200,1.0,NaN,False,0.186782
